In [114]:
import sapscript as ss
import pyperclip
import numpy as np
from datetime import datetime
from datetime import timedelta

In [159]:
E_PLANT = '7030' #ссылочный завод
E_STORAGE = '0023' #склад для агента RED
AGENT_ID = 'RED'
PARAMS = {'ERD':{'plant':'2059',
                 'agent':'100001589',  # 100001589 RED или 100004346 CSE
                 'idoc_port': 'SAPERD'}, 
          'ERT':{'plant':'3024',
                 'agent':'100001514', # 100001514 RED или 100004238 CSE
                 'idoc_port': 'SAPERT'}} 
ORDERS_CREATED_FILE = 'orders_created.txt' # берем отсюда номера ZIMG заказов, по которым планируем делать возвраты
RETURN_EXT_NUM_FILE = 'return_ext_num.txt' # счетчик внешнего номера возврата от Агента
TODAY = datetime.now().strftime('%Y%m%d')
TOMORROW = (datetime.now() + timedelta(days=1)).strftime('%Y%m%d')

class Returns:
    def __init__(self, _system_id):
        self.sys_id = _system_id
        self.sap = ss.sap()
        self.sap.check_system(self.sys_id)
        self.ext_number = [] # Внешние номера возврата Агента
        self.orders_created = [] # ZIMG заказы, участвующие в возвратах
        
    def zim_reestr(self):
        try:
            self.sap.run('zim_reestr')
            session = self.sap.session()
            session.findById("wnd[0]/usr/ctxtP_WERKS").text = E_PLANT
            session.findById("wnd[0]/usr/ctxtS_LGORT-LOW").text = E_STORAGE
            session.findById("wnd[0]/usr/ctxtS_WRKOUT-LOW").text = PARAMS[self.sys_id]['plant']
            
            # считываем созданные ZIMG заказы, по которым планируем делать возвраты
            with open(ORDERS_CREATED_FILE, 'r') as file:
                self.orders_created = file.read().split('\n')
                self.orders_created.remove('')
            if len(self.orders_created)==0:
                raise ValueError('В файле не указаны ZIMG заказы.')
            
            print('ZIMG заказы для возвратов:')
            for order_num in self.orders_created:
                print(order_num)
            # Вставляем из буфера заказы
            pyperclip.copy('\r\n'.join(self.orders_created))
            session.findById("wnd[0]/usr/btn%_S_KDAUF_%_APP_%-VALU_PUSH").press()
            session.findById("wnd[1]/tbar[0]/btn[16]").press()
            session.findById("wnd[1]/tbar[0]/btn[24]").press()
            session.findById("wnd[1]/tbar[0]/btn[8]").press()
            
            session.findById("wnd[0]").sendVKey(8)
            
            # считываем E-запасы по указанным заказам
            grid = session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell")
            stocks = self.sap.read_alv(grid, cols = ["MAT_KDAUF", "MATNR", "MENGE"], tech_names=True)
            stocks.drop(index=stocks[stocks['MAT_KDAUF']==''].index, inplace=True)
            stocks = stocks.sort_values(by='MAT_KDAUF', ascending=False)
            return stocks
        except:
            self.sap.error_print()
        finally:
            session = None
    
    def we19(self, e_stock, quantity='Full'):
        """Создание айдока ZIMRETAG
        Параметры:
        _e_stock - датафрейм из метода zim_reestr
        quantity - параметр, отвечающий за кол-во в возврате:
                   'Full'   - полный возврат всех позиций;
                   'Random' - случайное число, не превышающее кол-во из заказа;
                   'One'    - единица"""
        # Читаем текущий и записываем новый внешний номер возврата Агента из файла
        with open(RETURN_EXT_NUM_FILE, 'r') as file:
            ext_num = file.read().split('_')
            self.ext_number.append('_'.join(ext_num))
            ext_num[2] = str(int(ext_num[2])+1)
            ext_num_new = '_'.join(ext_num)
        with open(RETURN_EXT_NUM_FILE, 'w') as file:
            file.write(ext_num_new)
            
        try:
            self.sap.run('we19')
            session = self.sap.session()
            session.findById("wnd[0]/usr/radMSED7START-SEL_IDOCTP").select()
            session.findById("wnd[0]/usr/ctxtMSED7START-IDOCTYP").text = "ZIMRETAG"
            session.findById("wnd[0]/usr/ctxtMSED7START-IDOCTYP").setFocus()
            session.findById("wnd[0]/usr/ctxtMSED7START-IDOCTYP").caretPosition = 8
            session.findById("wnd[0]/tbar[1]/btn[8]").press()
            session.findById("wnd[0]/usr/lbl[9,1]").setFocus()
            session.findById("wnd[0]/usr/lbl[9,1]").caretPosition = 7
            session.findById("wnd[0]").sendVKey(2)
            session.findById("wnd[1]/usr/ctxtG_CONTROL_RECORD-RCVPOR").text = PARAMS[self.sys_id]['idoc_port']
            session.findById("wnd[1]/usr/ctxtG_CONTROL_RECORD-RCVPRN").text = PARAMS[self.sys_id]['agent']
            session.findById("wnd[1]/usr/ctxtEDI_PRT-RCVPRT").text = "LI"
            session.findById("wnd[1]/usr/ctxtG_CONTROL_RECORD-SNDPOR").text = PARAMS[self.sys_id]['idoc_port']
            session.findById("wnd[1]/usr/ctxtG_CONTROL_RECORD-SNDPRN").text = PARAMS[self.sys_id]['agent']
            session.findById("wnd[1]/usr/ctxtEDI_PRT-SNDPRT").text = "LI"
            session.findById("wnd[1]/usr/ctxtEDI_MSGTYP-MSGTYP").text = "ZIMRETAG"
            session.findById("wnd[1]/usr/ctxtEDI_MSGTYP-MSGTYP").setFocus()
            session.findById("wnd[1]/usr/ctxtEDI_MSGTYP-MSGTYP").caretPosition = 8
            session.findById("wnd[1]/tbar[0]/btn[0]").press()
            session.findById("wnd[0]/usr/lbl[36,3]").setFocus()
            session.findById("wnd[0]/usr/lbl[36,3]").caretPosition = 6
            
            session.findById("wnd[0]").sendVKey(2)
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[0,21]").text = AGENT_ID
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[1,21]").text = TODAY
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[2,21]").text = self.ext_number[0]
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[3,21]").text = TOMORROW
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[4,21]").text = PARAMS[self.sys_id]['plant']
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[4,21]").setFocus()
            session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[4,21]").caretPosition = 4
            session.findById("wnd[1]/tbar[0]/btn[0]").press()
            
            # Копируем сегменты для позиций
            for pos in range((len(e_stock)-1)):
                session.findById("wnd[0]/tbar[1]/btn[5]").press()
                session.findById("wnd[1]/usr/txt[5,15]").text = "ZIMRETAG_POSITION"
                session.findById("wnd[1]/usr/txt[5,15]").setFocus()
                session.findById("wnd[1]/usr/txt[5,15]").caretPosition = 0
                session.findById("wnd[1]/tbar[0]/btn[0]").press()
                
            # Заполняем данные сегментов по позициям
            for index in e_stock.index:
                # вычисляем количество возврата для позиции в зависимости от режима работы
                if quantity == 'Full':
                    menge = e_stock.iloc[index]['MENGE']
                elif quantity == 'One':
                    menge = 1
                elif quantity == 'Random':
                    menge = np.random.randint(0, e_stock.iloc[index]['MENGE']+1)
                    if menge == 0:
                        continue
                session.findById("wnd[0]/usr/lbl[36,"+str(index+4)+"]").setFocus()
                session.findById("wnd[0]/usr/lbl[36,"+str(index+4)+"]").caretPosition = 0
                session.findById("wnd[0]").sendVKey(2)    
                session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[0,21]").text = e_stock.iloc[index]['MAT_KDAUF']
                session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[1,21]").text = e_stock.iloc[index]['MATNR']
                session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[2,21]").text = menge
                session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[2,21]").setFocus()
                session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/txtSVALD-VALUE[2,21]").caretPosition = 1
                session.findById("wnd[1]/tbar[0]/btn[0]").press()
                
            # Отправляем айдок в порт
            session.findById("wnd[0]/tbar[1]/btn[8]").press()
            session.findById("wnd[1]/tbar[0]/btn[0]").press()
            message = session.findById("/app/con[0]/ses[0]/wnd[2]/usr/txtMESSTXT1").text
            session.findById("wnd[2]/tbar[0]/btn[0]").press()
            
            print('WE19: Внешний номер возврата Агента -', self.ext_number[0])
            print('WE19:', message)
        except:
            self.sap.error_print()
        finally:
            session = None
            
    def zimr(self, ext_num=None):
        try:
            self.sap.run('zimr')
            session = self.sap.session()
            session.findById("wnd[0]/usr/ctxtS_WRK-LOW").text = PARAMS[self.sys_id]['plant']
            
            # Указываем внешние номера заявок на возврат, если они есть
            if ext_num != None:
                pyperclip.copy('\r\n'.join(ext_num))
                session.findById("wnd[0]/usr/btn%_S_BOLNR_%_APP_%-VALU_PUSH").press()
                session.findById("wnd[1]/tbar[0]/btn[24]").press()
                session.findById("wnd[1]/tbar[0]/btn[8]").press()
            session.findById("wnd[0]/tbar[1]/btn[8]").press()
        except:
            self.sap.error_print()
        finally:
            session = None

## Возвраты в ERD

In [160]:
ret = Returns('ERD')

# Считываем запасы агента по ранее созданным и отгруженным заказам
e_stock = ret.zim_reestr()

# Создаем IDOC вида ZIMRETAG
ret.we19(e_stock, quantity='Random')

# Открываем ZIMR со списком созданных заявок на возврат
ret.zimr(ext_num=ret.ext_number)

ZIMG заказы для возвратов:
45961812
WE19: Внешний номер возврата Агента - EXT_NUM_10016
WE19: Номер IDOC 0000000001392053 передан в приложение.


In [161]:
ret.zimr(ext_num=ret.ext_number)

In [145]:
len(ret.orders_created)

1